In [1]:
import os
import sys
import subprocess
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import torch
from IPython.display import *


from pacer import (
    # read_dat_file,
    DatVersion,
    RawGPSSource,
    GPMFSource,
    SequentialGPSSource,
    CoordinateSystem,
    GPSSample,
    Vec3f,
    PointInTime_GPSSample,
    Point,
    Laps,
    Segment,
    Lap,
)

In [2]:
files = [
    "/Users/denys/Pictures/GH010251.MP4",
    "/Users/denys/Pictures/GH020251.MP4",
    "/Users/denys/Pictures/GH030251.MP4",
]

gpmf = GPMFSource(files[0])
# for f in files[1:]:
#     gpmf = SequentialGPSSource(gpmf, GPMFSource(f))

gpmf.get_total_duration()
samples = []


def on_sample(s: GPSSample, _, _2):
    if s.full_speed > 3:
        samples.append((s, gpmf.current_time_span()))


while not gpmf.is_end():
    gpmf.read_samples(on_sample)
    gpmf.next()

In [3]:
s1, s2 = samples[0][0], samples[531][0]
print(f"Sample 1: {s1}")
print(f"Sample 2: {s2}")

Sample 1: GPSSample(lat=52.040102, lon=-0.784965, altitude=77.683000, full_speed=3.001000, ground_speed=2.820000)
Sample 2: GPSSample(lat=52.039581, lon=-0.782842, altitude=78.377000, full_speed=18.865000, ground_speed=18.910000)


In [4]:
cs = CoordinateSystem(s1)

In [5]:
rough_frequency = len(samples) / len(set(span for _, span in samples))

data = np.array(
    [
        cs.distance(s1, s2)
        / (0.5 * s1.full_speed + 0.5 * s2.full_speed)
        * rough_frequency
        for (s1, _), (s2, _) in zip(samples[:-1], samples[1:])
    ]
)

px.scatter(data)

In [6]:
di = np.round(
    np.array(
        [
            cs.distance(s1, s2)
            / (0.5 * s1.full_speed + 0.5 * s2.full_speed)
            * rough_frequency
            for (s1, _), (s2, _) in zip(samples[:-1], samples[1:])
        ]
    )
)

px.scatter(di)

In [10]:
floor = torch.Tensor([b for (_, (b, _)) in samples])
ceil = torch.Tensor([e for (_, (_, e)) in samples])
di = np.round(
    np.array(
        [
            cs.distance(s1, s2)
            / (0.5 * s1.full_speed + 0.5 * s2.full_speed)
            * rough_frequency
            for (s1, _), (s2, _) in zip(samples[:-1], samples[1:])
        ]
    )
)
di = torch.Tensor(np.concatenate([[1], di]))

assert ceil.shape == floor.shape == di.shape


def loss(x):
    assert x.shape == di.shape, f"Expected {di.shape}, got {x.shape}"

    my_diffs = x[1:] - x[:-1]
    my_diffs /= di[1:]
    spacing = ((my_diffs - my_diffs.mean()) ** 2).mean()
    constraints = (((floor - x).clip(min=0) + (x - ceil).clip(min=0)) ** 2).mean()
    return spacing + constraints

In [11]:
t1 = (floor + ceil) / 2
t1.requires_grad_()

learning_curve = []

for lr in [1e-1, 1e-2, 1e-3]:
    optimizer = torch.optim.Adam([t1], lr=lr)
    for i in range(100):
        optimizer.zero_grad()
        l = loss(t1)
        l.backward()
        optimizer.step()
        learning_curve.append((lr, i, l.item()))
px.line(
    pd.DataFrame(learning_curve, columns=["lr", "iteration", "loss"]),
    y="loss",
    log_y=True,
    color="lr",
    title="Learning curve",
)

In [12]:
phase = torch.tensor(floor[0], requires_grad=True)
frequency = torch.tensor(rough_frequency, requires_grad=True)

t2 = phase + 1 / frequency * (di.long().cumsum(0).float() - 1)
learning_curve = []

for lr in [1e-1, 1e-2, 1e-3]:
    optimizer = torch.optim.Adam([phase, frequency], lr=lr)
    for i in range(100):
        optimizer.zero_grad()
        l = loss(t2)
        l.backward(retain_graph=True)
        optimizer.step()
        t2 = phase + 1 / frequency * (di.long().cumsum(0).float() - 1)
        learning_curve.append((lr, i, l.item()))

px.line(
    pd.DataFrame(learning_curve, columns=["lr", "iteration", "loss"]),
    y="loss",
    log_y=True,
    color="lr",
    title="Learning curve",
)

/var/folders/1h/jqy0k9wj1sx7s89lp8_kgm_r0000gn/T/ipykernel_48962/2896287337.py:1: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).



In [13]:
implied_frequency = di[1:] / (t1[1:] - t1[:-1])
px.scatter(implied_frequency.detach().numpy(), title="Implied frequency from GPS data")

In [14]:
implied_frequency = di[1:] / (t2[1:] - t2[:-1])
px.scatter(implied_frequency.detach().numpy(), title="Implied frequency from GPS data")

In [15]:
laps = Laps()
laps.set_coordinate_system(cs)

for (s, span), t in zip(samples, t1):
    laps.add_point(s, t)

s = laps.pick_random_start()
laps.sectors.start_line = s
laps.update()

pd.DataFrame([dict(lap=i, lap_time=laps.lap_time(i)) for i in range(laps.laps_count())])

,lap,lap_time
0,0,71.481822
1,1,70.374183
2,2,69.889058
3,3,69.039946
4,4,68.854761
5,5,70.615707
6,6,0.000000


In [16]:
delta_by_lap = []
for i in range(1, laps.laps_count()):
    reference_lap = laps.get_lap(i)
    reference_lap.width = 5
    lap0 = reference_lap.resample(laps.get_lap(3), cs)
    lap1 = reference_lap.resample(laps.get_lap(4), cs)

    t0 = (
        np.array([lap0.points[i].time for i in range(len(lap0.points))])
        - lap0.points[0].time
    )
    t1 = (
        np.array([lap1.points[i].time for i in range(len(lap1.points))])
        - lap1.points[0].time
    )
    delta = t1 - t0

    delta_by_lap.append(
        pd.DataFrame(
            dict(
                distance=lap0.cum_distances,
                delta=delta,
                reference_lap=f"Lap {i}",
            )
        )
    )

delta_by_lap = pd.concat(delta_by_lap)

px.line(
    delta_by_lap,
    x="distance",
    y="delta",
    color="reference_lap",
)

In [17]:
delta_by_lap = []
reference_lap = laps.get_lap(1)

for i in range(1, laps.laps_count() - 1):
    reference_lap.width = 5
    lap0 = reference_lap.resample(laps.get_lap(i), cs)
    lap1 = reference_lap.resample(laps.get_lap(5), cs)

    t0 = (
        np.array([lap0.points[i].time for i in range(len(lap0.points))])
        - lap0.points[0].time
    )
    t1 = (
        np.array([lap1.points[i].time for i in range(len(lap1.points))])
        - lap1.points[0].time
    )

    delta = t1 - t0

    delta_by_lap.append(
        pd.DataFrame(
            dict(
                lat=[lap0.points[i].point.lat for i in range(len(lap0.points))],
                lon=[lap0.points[i].point.lon for i in range(len(lap0.points))],
                distance=lap0.cum_distances,
                delta=delta,
                lap=i,
            )
        )
    )

delta_by_lap = pd.concat(delta_by_lap)

px.line(
    delta_by_lap,
    x="distance",
    y="delta",
    color="lap",
)

In [18]:
ddelta = delta_by_lap["delta"].diff()
ddelta = ddelta.clip(
    lower=delta.mean() - 2 * ddelta.std(), upper=ddelta.mean() + 2 * ddelta.std()
).rolling(20).mean()


px.scatter_map(
    delta_by_lap.assign(ddelta=ddelta).loc[lambda d: d["lap"] == 1],
    lat="lat",
    lon="lon",
    color="ddelta",
    zoom=17,
).update_layout(height=800)

In [19]:
px.histogram(delta_by_lap["delta"], nbins=200)

In [ ]:
lap = laps.get_lap(1)


def build_lap_df(lap):
    return pd.DataFrame(
        [
            dict(
                lat=s.point.latitude,
                lon=s.point.longitude,
                time=s.time - lap.points[0].time,
                distance=lap.cum_distances[i],
                i_point=i,
            )
            for i in range(lap.count())
            if (s := lap.points[i]) is not None
        ]
    )


all_laps = pd.concat(
    [build_lap_df(laps.get_lap(i)).assign(i_lap=i) for i in range(laps.laps_count())]
)


In [17]:
fig = px.scatter_map(
    all_laps,
    lat="lat",
    lon="lon",
    color="distance",
    hover_data=["i_lap", "i_point"],
    # map_style="basic",
    zoom=17,
)
fig.update_layout(height=800)


In [ ]:
start_line = laps.sectors.start_line
p1, p2 = start_line.first, start_line.second

In [ ]:
s1, s2 = map(lambda p: getattr(cs, "global")(Vec3f(p.x, p.y, 0)), (p1, p2))

In [ ]:
def add_segment(fig: go.Figure, s1: GPSSample, s2: GPSSample, name: str):
    fig.add_trace(
        go.Scattermap(
            mode="lines",
            lon=[s1.longitude, s2.longitude],
            lat=[s1.latitude, s2.latitude],
            name=name,
        )
    )
    return fig


add_segment(fig, s1, s2, "start_line")

In [ ]:
Point(1, 2)

Point(x=1.0, y=2.0)

In [ ]:
start_line

Segment(first=Point(x=83.9594233828202, y=-26.0512637901178), second=Point(x=75.5699800474182, y=-31.49343619845611))

In [ ]:
def to_point(s: GPSSample):
    v = getattr(cs, "local")(s)
    return Point(v[0], v[1])


first_seg = Segment(
    to_point(laps.get_lap(0).points[0].point), to_point(laps.get_lap(0).points[1].point)
)

add_segment(
    fig, laps.get_lap(0).points[0].point, laps.get_lap(0).points[1].point, "first_one"
)

In [ ]:
ratio, _ = (
    start_line.intersects(first_seg.first, first_seg.second),
    start_line.intersects(first_seg.second, first_seg.first),
)

In [ ]:
df = pd.DataFrame(
    dict(
        x=[
            start_line.first.x,
            start_line.second.x,
            None,
            first_seg.first.x,
            first_seg.second.x,
        ],
        y=[
            start_line.first.y,
            start_line.second.y,
            None,
            first_seg.first.y,
            first_seg.second.y,
        ],
        name=[1, 1, None, 2, 3],
    )
)

fig = px.line(df, x="x", y="y", hover_data="name")

In [ ]:
x, y = start_line.first, start_line.second
a, b = first_seg.first, first_seg.second

In [ ]:
res = (1 - ratio) * a + ratio * b
fig.add_trace(go.Scatter(x=[res.x], y=[res.y]))

In [ ]:
def rot(x):
    return Point(-x.y, x.x)

In [ ]:
n = rot(x - y)
n.scalar(a - x), n.scalar(b - x), n.scalar(a - y), n.scalar(b - y)

(-4.940411947284312,
 0.8100397188352676,
 -4.940411947284313,
 0.8100397188352667)

In [ ]:
a

Point(x=79.49651065482897, y=-29.535207990925674)

In [ ]:
dir(start_line.first)

['__add__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmul__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 'scalar',
 'x',
 'y']

In [ ]:
start_line.first.scalar(start_line.first)

7727.85311983796

In [ ]:
start_line.intersects(first_seg.first, first_seg.second, None)

TypeError: intersects(): incompatible function arguments. The following argument types are supported:
    1. intersects(self, fst: _pacer_geometry_impl.Point, snd: _pacer_geometry_impl.Point) -> float | None

Invoked with types: _pacer_laps.Segment, _pacer_geometry_impl.Point, _pacer_geometry_impl.Point, NoneType

In [ ]:
cs.global_()

AttributeError: '_pacer_geometry_impl.CoordinateSystem' object has no attribute 'global_'

In [ ]:
px.scatter_map(
    points_df,
    lat="latitude",
    lon="longitude",
    color="speed",
    map_style="outdoors",
).update_layout(height=800)

NameError: name 'points_df' is not defined

In [ ]:
lap = laps.get_lap(1)
dir(lap)

In [ ]:
lap.points

In [ ]:
delta_by_lap

In [ ]:
px.scatter(delta[1:] - delta[:-1])

In [ ]:
c = 0.1
noise = np.round((delta[1:] - delta[:-1]) / c) * c
px.scatter(noise)

In [ ]:
px.scatter(np.cumsum(noise), title="Cumulative noise")

In [ ]:
px.line(delta - np.concatenate([[0], np.cumsum(noise)]))

In [ ]:
laps = Laps()
laps.set_coordinate_system(cs)

for (s, span), t in zip(samples, t2):
    laps.add_point(s, t)
s = laps.pick_random_start()
laps.sectors.start_line = s
laps.update()
reference_lap = laps.get_lap(1)
reference_lap.width = 5
lap0 = reference_lap.resample(laps.get_lap(3), cs)
lap1 = reference_lap.resample(laps.get_lap(4), cs)
px.line(
    [
        lap1.points[i].time
        - lap0.points[i].time
        - lap1.points[0].time
        + lap0.points[0].time
        for i in range(len(lap0.points))
    ]
)